In [1]:
import sys
import cv2
import os
from sys import platform
import argparse
import time
import numpy as np
import matplotlib.pyplot as plt
import pyopenpose as op
import pandas as pd

In [2]:
# Custom Params (refer to include/openpose/flags.hpp for more parameters)
params = dict()
params["model_folder"] = "/home/ubuntu/openpose/models"

In [3]:
# Starting OpenPose
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()

input_image_path="/home/ubuntu/openpose/examples/media/"
output_image_path="/home/ubuntu/output_images/"
output_keypoints_path= "/home/ubuntu/output_keypoints/"

# Read frames on directory
imagePaths = op.get_images_on_directory(input_image_path)
    #"/Users/gwattal/squatsmart/openpose/examples/media")
start = time.time()

In [ ]:
for imagePath in imagePaths:
    image_name=imagePath.replace(input_image_path,"")
    image_name=image_name.replace(".jpg",'')
    print(image_name)
    datum = op.Datum()
    imageToProcess = cv2.imread(imagePath)
    datum.cvInputData = imageToProcess
    opWrapper.emplaceAndPop([datum])
    ##Save the rendered output file
    cv2.imwrite(output_image_path+image_name+"_annotated.jpg",datum.cvOutputData)
    #Save the Keypoints
    #np.savetxt(output_keypoints_path+image_name+"_keypoints.txt",datum.poseKeypoints)#.reshape(25,3))
    #print("Body keypoints: \n" + str(datum.poseKeypoints))
    print("Done with " + image_name)

COCO_val2014_000000000192
Done with COCO_val2014_000000000192
COCO_val2014_000000000241


In [ ]:
opWrapper.stop()

In [ ]:
end = time.time()
print(end - start)

In [8]:
path="/Users/gwattal/squatsmart/output_keypoints/test_output_keypoints/"
entries = os.listdir(path)

In [9]:
entries

['test_output_keypointsIMG_6107.JPG_keypoints.txt',
 'test_output_keypointsIMG_6106.JPG_keypoints.txt',
 'test_output_keypointsIMG_6105_keypoints.txt']

In [12]:
name=[]
stance_heel=[]
stance_ankle=[]
stance_wgt_avg=[]
confidence_heel=[]
confidence_ankle=[]
confidence_wgt_avg=[]
for file in entries:
    #print(file)
    x,y,c=np.loadtxt((path+file)).T
    ##
    r_shoulder_x=x[2]
    r_shoulder_y=y[2]
    r_shoulder_c=c[2]
    r_heel_x= x[24]
    r_heel_y= y[24]
    r_heel_c= c[24]
    r_ankle_x=x[11]
    r_ankle_y=y[11]
    r_ankle_c=c[11]
    l_shoulder_x=x[5]
    l_shoulder_y=y[5]
    l_shoulder_c=c[5]
    l_heel_x= x[21]
    l_heel_y= y[21]
    l_heel_c= c[21]
    l_ankle_x=x[14]
    l_ankle_y=y[14]
    l_ankle_c=c[14]
    r_wgt_avg_x=(r_heel_x*r_heel_c + r_ankle_x*r_ankle_c)/(r_heel_c + r_ankle_c) #weighted avg of heel and ankle
    r_wgt_avg_c=2*(r_heel_c*r_ankle_c)/(r_heel_c+r_ankle_c) #confidence is harmonic mean of individual confidences
    l_wgt_avg_x=(l_heel_x*l_heel_c + l_ankle_x*l_ankle_c)/(l_heel_c + l_ankle_c)
    l_wgt_avg_c=2*(l_heel_c*l_ankle_c)/(l_heel_c+l_ankle_c) #confidence is harmonic mean of individual confidences
    ##
    name.append(file.replace("_keypoints.txt",""))
    #stance calculated from heel
    stance_heel.append((l_heel_x-r_heel_x)/(l_shoulder_x - r_shoulder_x))
    confidence_heel.append(min(r_shoulder_c,r_heel_c,l_shoulder_c,l_heel_c))
    ##
    #stance calculated from ankle
    stance_ankle.append((l_ankle_x-r_ankle_x)/(l_shoulder_x - r_shoulder_x))
    confidence_ankle.append(min(r_shoulder_c,r_ankle_c,l_shoulder_c,l_ankle_c))
    ##
    #stance calculated form weighted avg
    stance_wgt_avg.append((l_wgt_avg_x-r_wgt_avg_x)/(l_shoulder_x - r_shoulder_x))
    confidence_wgt_avg.append(min(r_shoulder_c,r_wgt_avg_c,l_shoulder_c,l_wgt_avg_c))
    #print(file,(l_heel_x-r_heel_x)/(l_shoulder_x - r_shoulder_x))
df = pd.DataFrame({'name': name, 'stance_heel': stance_heel, "confidence_min_heel":confidence_heel,
                   'stance_ankle': stance_ankle, "confidence_min_ankle":confidence_ankle,
                  'stance_wgt_avg': stance_wgt_avg, "confidence_min_wgt_avg":confidence_wgt_avg})
#df=df.sort_values(by=['stance'])
print("done")

done


In [13]:
df

,name,stance_heel,confidence_min_heel,stance_ankle,confidence_min_ankle,stance_wgt_avg,confidence_min_wgt_avg
0,test_output_keypointsIMG_6107.JPG,1.427969,0.625727,1.551691,0.808752,1.491509,0.722797
1,test_output_keypointsIMG_6106.JPG,1.070755,0.559829,1.238220,0.715353,1.163259,0.628107
2,test_output_keypointsIMG_6105,1.016248,0.746233,1.236688,0.785347,1.129869,0.765291


In [28]:
datum.poseKeypoints

array([[[6.0786536e+02, 3.3551569e+02, 9.3835557e-01],
        [6.3915863e+02, 3.5704959e+02, 7.9298127e-01],
        [6.3334192e+02, 3.5701880e+02, 7.8994620e-01],
        [6.1570996e+02, 3.9822754e+02, 4.9364272e-01],
        [5.9017480e+02, 4.1782666e+02, 5.1929474e-01],
        [6.4508478e+02, 3.5706418e+02, 9.0333217e-01],
        [6.3525861e+02, 4.0602167e+02, 9.3825132e-01],
        [5.9217584e+02, 4.1976962e+02, 8.9142793e-01],
        [6.3129584e+02, 4.3352115e+02, 7.7044481e-01],
        [6.2350568e+02, 4.3148474e+02, 7.4644285e-01],
        [5.6474573e+02, 4.3539462e+02, 6.0213131e-01],
        [5.1961700e+02, 4.9808640e+02, 5.7077074e-01],
        [6.3532611e+02, 4.3738162e+02, 8.2645738e-01],
        [5.7064673e+02, 4.4526376e+02, 8.8986588e-01],
        [5.1965588e+02, 5.0005600e+02, 7.7072769e-01],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [6.1170850e+02, 3.2960275e+02, 8.9180070e-01],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [6

In [29]:
datum.poseKeypoints[2,9,:]

array([237.56035  , 500.08224  ,   0.6383891], dtype=float32)

In [30]:
datum.poseKeypoints[2,12,:]

array([2.6890405e+02, 4.9615585e+02, 4.6890140e-01], dtype=float32)

In [31]:
datum.poseKeypoints[2,4,:]

array([2.5521547e+02, 4.0998798e+02, 1.5201777e-01], dtype=float32)

In [32]:
datum.poseKeypoints[2,7,:]

array([3.8839133e+02, 4.3937634e+02, 1.2872599e-01], dtype=float32)

In [33]:
datum.poseKeypoints[2,:,2]

array([0.8992834 , 0.6029232 , 0.50271344, 0.14340797, 0.15201777,
       0.46852618, 0.14418294, 0.12872599, 0.57785606, 0.6383891 ,
       0.8309093 , 0.86399543, 0.4689014 , 0.4286189 , 0.544904  ,
       0.8446512 , 0.05263789, 0.81748104, 0.        , 0.54739356,
       0.48929518, 0.12346784, 0.69349813, 0.68978375, 0.8532623 ],
      dtype=float32)

In [34]:
datum.poseKeypoints[2,:,0]

array([351.19174, 304.1929 , 286.5493 , 239.4955 , 255.21547, 323.77557,
       315.94788, 388.39133, 253.19344, 237.56035, 331.58414, 274.7596 ,
       268.90405, 339.42416, 292.37067, 341.40186, 349.19174, 317.8433 ,
         0.     , 323.80026, 321.80307, 282.6442 , 315.8922 , 300.2842 ,
       261.07166], dtype=float32)